In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from Module.Preprocessing import *
from Module.Stat import *

#시각화
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import *
from sklearn.metrics import accuracy_score, confusion_matrix, log_loss

##알고리즘 
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

# 임시로 X_train, X_test, y_train, y_test 만들기
# 내가 받을때는 필요한거만 전처리되어서 온다는 전제
# k_fold로 임시적으로 돌아가는지 test

In [2]:
df = pd.read_csv("CleanData/X_train_After_IPO_1.csv",encoding = 'cp949')
df.drop(['종목명','상장일','공모 시가총액'],axis = 1,inplace = True)

In [3]:
X = df.drop('1_3_수익률',axis=1)
y = Cut(df['1_3_수익률'],[-0.2, -0.1, 0.1, 0.2])

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y,\
                                                    test_size=0.2)

# 여기서부터 인풋 받았다 생각하고 하기

# 여기서부터 재성이가 보면 됨 ㅇㅇ
# K_fold 부분은 나중에 X,y List로 대체되어서 반복해야함
# 아래는 그냥 재성이가 보면 되는 대충의 예시 for 문 에서 나온 list가 데이터 프레임으로 만들어지는 거만 확인하면 ㅇㅋ

In [5]:
kfold = KFold(n_splits = 5)
cv_accuracy = []
n_iter = 0 

In [6]:
"""
0. 사실상 Fit에 해당하는 부분임

1. X_train, X_test y_train, y_test 받는다는 전제하에 짠 것임

2. 예상 Output
(1) y_test_pred를 리턴해서 재성이가 만든 쪽에서 y_수익률,-> 가공된 y_test_label,->예측된 y_test_pred \
만들 수 있도록 해줘야함

3. 재성이가 말한 문제를 해결은 했으나 recall 에서는 weighted로 IR을 고려해도 acc와 동일하게 나오는 문제가 있음 F1만 보여주던가 해야할 듯
"""

score_list = []
feature_list = []
for train_idx, test_idx in kfold.split(X):
    # 결과 엑셀 담기
    X_train, X_test = X.loc[train_idx,:], X.loc[test_idx,:]
    y_train, y_test = y[train_idx], y[test_idx]

    rf_clf = RandomForestClassifier(random_state=0)
    rf_clf.fit(X_train , y_train)
    train_pred = rf_clf.predict(X_train)
    test_pred = rf_clf.predict(X_test)

    # 성과 평가
    train_res = get_clf_eval(y_train, train_pred)
    test_res = get_clf_eval(y_test,test_pred)
    res = train_res + test_res

    #
    score_list.append(res)
    feature_list.append(rf_clf.feature_importances_)


# 나중에 통계낼때는 groupby 활용 & index는 train 시작 날짜로 해야함
score_df = pd.DataFrame(score_list, columns = ["Train"]*4+["Test"]*4).T 

feature_df = pd.DataFrame(feature_list, columns = X_train.columns).T



정확도(accuracy): 1.0000, 정밀도(precision): 1.0000, 재현율(recall): 1.0000, f1_score: 1.0000
정확도(accuracy): 1.0000, 정밀도(precision): 1.0000, 재현율(recall): 1.0000, f1_score: 1.0000
정확도(accuracy): 0.3023, 정밀도(precision): 0.2776, 재현율(recall): 0.3023, f1_score: 0.2286
정확도(accuracy): 0.3023, 정밀도(precision): 0.2173, 재현율(recall): 0.1899, f1_score: 0.1538
정확도(accuracy): 1.0000, 정밀도(precision): 1.0000, 재현율(recall): 1.0000, f1_score: 1.0000
정확도(accuracy): 1.0000, 정밀도(precision): 1.0000, 재현율(recall): 1.0000, f1_score: 1.0000
정확도(accuracy): 0.3488, 정밀도(precision): 0.2862, 재현율(recall): 0.3488, f1_score: 0.2709
정확도(accuracy): 0.3488, 정밀도(precision): 0.2109, 재현율(recall): 0.2282, f1_score: 0.1851
정확도(accuracy): 1.0000, 정밀도(precision): 1.0000, 재현율(recall): 1.0000, f1_score: 1.0000
정확도(accuracy): 1.0000, 정밀도(precision): 1.0000, 재현율(recall): 1.0000, f1_score: 1.0000
정확도(accuracy): 0.4844, 정밀도(precision): 0.4007, 재현율(recall): 0.4844, f1_score: 0.4192
정확도(accuracy): 0.4844, 정밀도(precision): 0.2983, 재현율(recall): 0.269

# 함수 사용 예시_Forest알고리즘을 받고 deepcopy하는 것만 수정이 필요

In [22]:
def Result(clf = None, X_train = None, X_test = None, y_train = None, y_test = None):
    # 반복문을 돌면서 모형이 지속적으로 fit되는 것을 방지하기 위해서 model의 deepcopy(?)가 필요함
    # 마찬가지로 하이퍼 파라미터를 맞춘 clf를 밖에서 선언하고 가져와야함
    clf = RandomForestClassifier() # clf = copy.deepcopy(clf)
    
    clf.fit(X_train , y_train)
    train_pred = clf.predict(X_train)
    test_pred = clf.predict(X_test)
    
    # 성과 평가
    train_res = get_clf_eval(y_train, train_pred)
    test_res = get_clf_eval(y_test,test_pred)
    res = train_res + test_res
    
    #feature_importance
    feature_importance = clf.feature_importances_
    
    D = {"Score":res, "Feature":feature_importance, "y_pred":test_pred}
    return D

In [23]:
rf_clf = RandomForestClassifier(random_state=0)
D = Result(rf_clf,X_train,X_test, y_train, y_test)

Confusion Matrix
[[ 86   0   0   0   0]
 [  0  71   0   0   0]
 [  0   0 211   0   0]
 [  0   0   0  59   0]
 [  0   0   0   0  87]]
정확도(accuracy): 1.0000, 정밀도(precision): 1.0000, 재현율(recall): 1.0000, f1_score: 1.0000
정확도(accuracy): 1.0000, 정밀도(precision): 1.0000, 재현율(recall): 1.0000, f1_score: 1.0000
Confusion Matrix
[[ 7  0 12  0  0]
 [12  1 16  0  0]
 [ 7  4 35  1  0]
 [ 1  1 11  0  0]
 [ 3  1 16  0  0]]
정확도(accuracy): 0.3359, 정밀도(precision): 0.3359, 재현율(recall): 0.3359, f1_score: 0.3359
정확도(accuracy): 0.3359, 정밀도(precision): 0.2098, 재현율(recall): 0.3359, f1_score: 0.2426


In [24]:
D["Score"]

[1.0,
 1.0,
 1.0,
 1.0,
 0.3359375,
 0.209796626984127,
 0.3359375,
 0.24261157093616031]

# 저장 예시
* 내 함수 밖에서 선언한 리스트에 내꺼에서 나온 것들을 append
* for 문이 끝나면 데이터 프레임 선언하고 저장

In [ ]:
    score_list.append(res)
    feature_list.append(rf_clf.feature_importances_)


# 나중에 통계낼때는 groupby 활용 & index는 train 시작 날짜로 해야함
score_df = pd.DataFrame(score_list, columns = ["Train"]*4+["Test"]*4).T 

feature_df = pd.DataFrame(feature_list, columns = X_train.columns).T

